In [ ]:
#Predicting the likelihood of getting admitted using Tensorflow
import app
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow	import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.metrics import r2_score
from tensorflow.keras.optimizers import Adam
#Loading the data into a pandas dataframe
adm_dataset = pd.read_csv("admissions_data.csv")
print(adm_dataset.head())
#Splitting the data into features and labels
features = adm_dataset.iloc[:, 0:8]
labels = adm_dataset.iloc[:, -1]
#Splitting into training and test sets
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.20, random_state = 23)
#Standardizing the numerical features
ct = ColumnTransformer([('scale', StandardScaler(), ["Serial No.", "GRE Score", "TOEFL Score", "University Rating", "SOP", "LOR ", "CGPA", "Research"])], remainder = 'passthrough')
features_train_scale = ct.fit_transform(features_train)
features_test_scale = ct.transform(features_test)
features_train_scale = pd.DataFrame(features_train_scale, columns = features_train.columns)
features_test_scale = pd.DataFrame(features_test_scale, columns = features_test.columns)
#Creating the model
adm_model = Sequential()
input = layers.InputLayer(input_shape = (8,))
adm_model.add(input)
adm_model.add(layers.Dense(2, activation = 'relu'))
adm_model.add(layers.Dense(1))
opt = Adam(learning_rate = 0.01)
adm_model.compile(loss = 'mse', metrics = ['mae'], optimizer = opt)
#Fitting and evaluating the model
history = adm_model.fit(features_train_scale, labels_train, epochs = 20, batch_size = 5, verbose = 1, validation_split = 0.2)
val_mse, val_mae = adm_model.evaluate(features_test, labels_test)
print(val_mse, val_mae)
# Do extensions code below
#Visualizing the loss for each epoch for both training and validation data
fig = plt.figure()
ax1 = fig.add_subplot(2, 1, 1)
ax1.plot(history.history['mae'])
ax1.plot(history.history['val_mae'])
ax1.set_title('model mae')
ax1.set_ylabel('MAE')
ax1.set_xlabel('epoch')
ax1.legend(['train', 'validation'], loc='upper left')

ax2 = fig.add_subplot(2, 1, 2)
ax2.plot(history.history['loss'])
ax2.plot(history.history['val_loss'])
ax2.set_title('model loss')
ax2.set_ylabel('loss')
ax2.set_xlabel('epoch')
ax2.legend(['train', 'validation'], loc='upper left')
fig.tight_layout()
# if you decide to do the Matplotlib extension, you must save your plot in the directory by uncommenting the line of code below

fig.savefig('static/images/my_plots.png')
#Evaluating how strongly the features predict an applicant’s admission using R-squared value
predicted_values = adm_model.predict(features_test_scale) 
print(r2_score(labels_test, predicted_values))